# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [73]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [74]:
current_dir = os.getcwd()
# This notebook is in public repro.
short_current_dir = current_dir[current_dir.find('SpringBoard'):] 
print('...\\'+short_current_dir)
file  = 'step3_output.csv'
if file in os.listdir('.\\data\\'):
    path=current_dir
    print(file+f' is in working dir')
else:
    print(file+f' is not in working dir. Change directory!')
df0 = pd.read_csv(f'{path}\\data\\step3_output.csv')
df0.head(5)

...\SpringBoard\Guided Capstone
step3_output.csv is in working dir


Name       state  summit_elev  vertical_drop  \
0               Hilltop Ski Area      Alaska         2090            294   
1            Sunrise Park Resort     Arizona        11100           1800   
2  Yosemite Ski & Snowboard Area  California         7800            600   
3               Donner Ski Ranch  California         8012            750   
4                  June Mountain  California        10090           2590   

   trams  fastEight  fastSixes  fastQuads  quad  triple  ...  \
0      0        0.0          0          0     0       1  ...   
1      0        0.0          0          1     2       3  ...   
2      0        0.0          0          0     0       1  ...   
3      0        0.0          0          0     0       1  ...   
4      0        0.0          0          2     0       0  ...   

   SkiableTerrain_ac  Snow Making_ac  daysOpenLastYear  yearsOpen  \
0               30.0       30.000000        150.000000       36.0   
1              800.0       80.000000        115.000000       49.0   
2               88.0      174.873239        110.000000       84.0   
3              505.0       60.000000        163.000000       82.0   
4             1500.0      174.873239        152.111111       58.0   

   averageSnowfall  AdultWeekday  AdultWeekend  projectedDaysOpen  \
0             69.0     30.000000      34.00000              152.0   
1            250.0     74.000000      78.00000              104.0   
2            300.0     47.000000      47.00000              107.0   
3            400.0     75.000000      75.00000              170.0   
4            250.0     57.916957      64.16681              128.0   

   NightSkiing_ac  clusters  
0            30.0         0  
1            80.0         1  
2             0.0         1  
3             0.0         1  
4             0.0         1  

[5 rows x 26 columns]

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [75]:
df = pd.get_dummies(df0, columns=['state'])
df.head(2)

Name  summit_elev  vertical_drop  trams  fastEight  \
0     Hilltop Ski Area         2090            294      0        0.0   
1  Sunrise Park Resort        11100           1800      0        0.0   

   fastSixes  fastQuads  quad  triple  double  ...  state_Rhode Island  \
0          0          0     0       1       0  ...                   0   
1          0          1     2       3       1  ...                   0   

   state_South Dakota  state_Tennessee  state_Utah  state_Vermont  \
0                   0                0           0              0   
1                   0                0           0              0   

   state_Virginia  state_Washington  state_West Virginia  state_Wisconsin  \
0               0                 0                    0                0   
1               0                 0                    0                0   

   state_Wyoming  
0              0  
1              0  

[2 rows x 60 columns]

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [76]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing as pp

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = pp.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [77]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [78]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [79]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = lm.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [80]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
from sklearn.metrics import mean_squared_error
sqrt = np.sqrt
rmse1 = sqrt(mean_squared_error(y_test, y_pred))
exvarscore1 = explained_variance_score(y_test, y_pred)
mean_abs_err1= mean_absolute_error(y_test, y_pred)
print(f' RMSE = {rmse}\n explained variance score = {exvarscore}\n mean absolute error = {mean_abs_err}  ')

 RMSE = 5.891099464955975
 explained variance score = 0.8184788934250462
 mean absolute error = 4.911071293639647  


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [81]:
print(model.intercept_)

56.92986962730687


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [82]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(
    by='Coefficient', ascending=False ).head(10)


Coefficient
fastEight       2.152012e+13
fastSixes       6.384626e+12
total_chairs    1.396031e+12
surface         7.927593e+11
double          7.515358e+11
triple          6.581906e+11
quad            3.970412e+11
fastQuads       3.167426e+11
state_New York  1.293249e+11
state_Michigan  1.164717e+11

**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [83]:
# We do not need the dummy features for the states this time
X2 = df0.drop(['Name','state' ,'AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y2 = df0.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler2 = pp.StandardScaler().fit(X2)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X2_scaled=scaler2.transform(X2) 

In [84]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y2 = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y2, test_size=0.25, random_state=1)

In [85]:
lm2 = linear_model.LinearRegression()
model2 = lm2.fit(X2_train,y2_train)

In [86]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y2_pred = lm2.predict(X2_test)
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
rmse2 = sqrt(mean_squared_error(y2_test, y2_pred))
exvarscore2 = explained_variance_score(y2_test, y2_pred)
mean_abs_err2= mean_absolute_error(y2_test, y2_pred)
print(f' RMSE = {rmse2}\n explained variance score = {exvarscore2}\n mean absolute error = {mean_abs_err2}  ')
print('Model Intercept = {}'.format(model.intercept_))
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm2.coef_), X2.columns, columns=['Coefficient']).sort_values(
    by='Coefficient', ascending=False )

 RMSE = 6.012204052076119
 explained variance score = 0.8035920854726438
 mean absolute error = 4.980343364189925  
Model Intercept = 56.92986962730687


Coefficient
AdultWeekday       1.080512e+01
summit_elev        3.037676e+00
clusters           2.501224e+00
daysOpenLastYear   1.675821e+00
vertical_drop      1.632381e+00
quad               1.496784e+00
averageSnowfall    1.441732e+00
surface            1.281581e+00
triple             1.160191e+00
Runs               1.098753e+00
double             9.450514e-01
projectedDaysOpen  8.831903e-01
SkiableTerrain_ac  8.556369e-01
Snow Making_ac     7.925827e-01
total_chairs       7.448284e-01
TerrainParks       6.168661e-01
LongestRun_mi      6.065388e-01
NightSkiing_ac     2.897607e-01
fastQuads          3.903336e-02
yearsOpen          1.825797e-02
trams              1.332268e-15
fastSixes          8.881784e-16
fastEight          6.661338e-16

**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

<font style="color:red; background-color:powderblue;"> <b>We already droped the base elev in step 3</b></font>

#### Model 3

In [87]:
# We do not need the dummy features for the states this time
X3 = df0.drop(['Name','state', 'summit_elev' ,'AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y3 = df0.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler3 = pp.StandardScaler().fit(X3)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X3_scaled=scaler3.transform(X3) 

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y3 = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X3_train, X3_test, y3_train, y3_test = train_test_split(X3_scaled, y3, test_size=0.25, random_state=1)

lm3 = linear_model.LinearRegression()
model3 = lm3.fit(X3_train,y3_train)
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y3_pred = lm3.predict(X3_test)
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
rmse3 = sqrt(mean_squared_error(y3_test, y3_pred))
exvarscore3 = explained_variance_score(y3_test, y3_pred)
mean_abs_err3= mean_absolute_error(y3_test, y3_pred)
print(f' RMSE = {rmse3}\n explained variance score = {exvarscore3}\n mean absolute error = {mean_abs_err3}  ')
print('Model Intercept = {}'.format(model.intercept_))
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm3.coef_), X3.columns, columns=['Coefficient']).sort_values(
    by='Coefficient', ascending=False ).head(10)

 RMSE = 6.415521631444835
 explained variance score = 0.7762273485537097
 mean absolute error = 5.329511571257056  
Model Intercept = 56.92986962730687


Coefficient
AdultWeekday         10.720058
averageSnowfall       2.220795
SkiableTerrain_ac     1.924572
clusters              1.871557
vertical_drop         1.523553
quad                  1.319474
double                1.209206
daysOpenLastYear      1.103282
surface               1.099534
Snow Making_ac        1.084775

#### Model 4 

In [98]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 26 22:21:41 2020

@author: kurt_
"""

#%%
# We do not need the dummy features for the states this time
X4 = df.drop(['Name', 'summit_elev' ,'AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y4 = df0.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler4 = pp.StandardScaler().fit(X4)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X4_scaled=scaler4.transform(X4) 

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y4 = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X4_train, X4_test, y4_train, y4_test = train_test_split(X4_scaled, y4, test_size=0.25, random_state=1)

lm4 = linear_model.LinearRegression()
model4 = lm4.fit(X4_train,y4_train)
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y4_pred = lm4.predict(X4_test)
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
rmse4 = sqrt(mean_squared_error(y4_test, y4_pred))
exvarscore4 = explained_variance_score(y4_test, y4_pred)
mean_abs_err4= mean_absolute_error(y4_test, y4_pred)
print(f' RMSE = {rmse4}\n explained variance score = {exvarscore4}\n mean absolute error = {mean_abs_err4}  ')
print('Model Intercept = {}'.format(model.intercept_))
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm4.coef_), X4.columns, columns=['Coefficient']).sort_values(
    by='Coefficient', ascending=False ).head(10)

 RMSE = 5.813252474993876
 explained variance score = 0.8265339490011624
 mean absolute error = 4.726327139935685  
Model Intercept = 56.92986962730687


Coefficient
fastEight       1.662508e+13
fastSixes       6.346705e+12
trams           6.211029e+12
total_chairs    4.576378e+12
surface         2.598773e+12
double          2.463636e+12
triple          2.157638e+12
quad            1.301555e+12
fastQuads       1.038325e+12
state_New York  1.395179e+10

In [110]:
for i in [1,2,3,4]:
    print(f'Model {i}')
    print(f' RMSE = {eval(f"rmse{i}")}\n explained variance score  = {eval(f"exvarscore{i}")}\n mean absolute error = {eval(f"mean_abs_err{i}")}  ')


Model 1
 RMSE = 5.891099464955975
 explained variance score  = 0.8184788934250462
 mean absolute error = 4.911071293639647  
Model 2
 RMSE = 6.012204052076119
 explained variance score  = 0.8035920854726438
 mean absolute error = 4.980343364189925  
Model 3
 RMSE = 6.415521631444835
 explained variance score  = 0.7762273485537097
 mean absolute error = 5.329511571257056  
Model 4
 RMSE = 5.813252474993876
 explained variance score  = 0.8265339490011624
 mean absolute error = 4.726327139935685  


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.82 | 4.91 |'base_elev'|
| Model 2. | 0.80|4.98 |'state', 'base_elev'|
| Model 3. | 0.78 | 5.33|'state','summit_elev','base_elev'|
| __Model 4.__|__0.83__ |__4.73__|__'summit_elev','base_elev'__|

Model Selection:

**Model 4**

Although it was not given as an option in the instructions I think that state is still an important feature to determine ticket price. Even though it is not an actionabal faeture, we build our model based on the data that consists of resorts all around the US. Hence, when the managers want to see what should their ticket price be so that it would be fair comparing to their competitors our model needs to account for where the resort is located. For instance if the ticket prices are usually cheaper in a state then without including state variable the model will likely predict higher than average price for that specific state. Likewise, the managers might get the wrong impression that their target price would be too high to be fair if their resort located in an expensive state for such resorts if model do not include state variable. 

Lastly, in order to meet the assumption of linear regression, our model should inlcude no or little multicollinearity. Therefore we need to get rid of those features which are highly correlated. However, since we already got rid of base_elev in the 3<sup>rd</sup> step we don't normally have to remove summit_elev. On the other hand, the model without it ( 4<sup>th</sup> model) performs better in terms of the metrics we use here. Therefore, among the Model 1 and Model 4 I choose the simplest one which is the Model 4